In [1]:
import pystow
import rich
import bioontologies
from bioontologies.obograph import _clean_uri, _compress_uri
import bioregistry
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import bioversions
from matplotlib_venn import venn2
from dataclasses import dataclass
import json
from textwrap import dedent
from functools import lru_cache
import biomappings
import pyobo
from tabulate import tabulate
from IPython.display import HTML
import time
import pandas as pd
import sys
from collections import defaultdict, Counter

In [2]:
print(sys.version)
print(time.asctime())

3.9.13 (main, May 24 2022, 21:28:31) 
[Clang 13.1.6 (clang-1316.0.21.2)]
Fri Aug  5 15:52:04 2022


In [3]:
EVALUATION = pystow.module("biomappings", "evaluation")
SIDER_URL = "http://sideeffects.embl.de/media/download/meddra_all_se.tsv.gz"


@dataclass
class Result:
    dataset: str
    source: str
    target: str
    total: int
    missing: int
    missing_biomappings: int
    missing_predictions: int

    @classmethod
    def make(
        cls,
        dataset,
        source,
        target,
        datasource_identifiers,
        primary,
        secondary,
        tertiary,
    ):
        return cls.from_dicts(
            dataset=dataset,
            source=source,
            target=target,
            datasource_identifiers=datasource_identifiers,
            ontology_external_identifiers=set(primary[source][target]),
            biomappings_external_identifiers=set(secondary.get(source, {}).get(target, {})),
            biomappings_prediction_identifiers=set(tertiary.get(source, {}).get(target, {})),
        )

    @classmethod
    def from_dicts(
        cls,
        dataset,
        source,
        target,
        datasource_identifiers,
        ontology_external_identifiers,
        biomappings_external_identifiers,
        biomappings_prediction_identifiers,
    ):
        return Result(
            dataset=dataset,
            source=source,
            target=target,
            total=len(datasource_identifiers),
            missing=len(datasource_identifiers - ontology_external_identifiers),
            missing_biomappings=len(
                datasource_identifiers
                - ontology_external_identifiers
                - biomappings_external_identifiers
            ),
            missing_predictions=len(
                datasource_identifiers
                - ontology_external_identifiers
                - biomappings_external_identifiers
                - biomappings_prediction_identifiers
            ),
        )

    def print(self):
        print(
            tabulate(
                [
                    (f"Total in {self.dataset}", f"{self.total:,}", ""),
                    (
                        f"Missing w/ {self.source}",
                        f"{self.missing:,}",
                        f"{self.missing / self.total:.2%}",
                    ),
                    (
                        f"Missing w/ {self.source} + BM.",
                        f"{self.missing_biomappings:,}",
                        f"{self.missing_biomappings / self.total:.1%}",
                    ),
                    (
                        f"Missing w/ {self.source} + BM. + Pred.",
                        f"{self.missing_predictions:,}",
                        f"{self.missing_predictions / self.total:.1%}",
                    ),
                ],
                headers=["Missing", f"Unmappable to {self.target}", "% Unmappable"]
            )
        )


evaluation_results = []

# Importing Mappings

In [4]:
# Primary mappings from OBO and other sources are going in here
primary_dd = defaultdict(dict)

summary_rows = []

## Biomappings

Manually curated mappings from Biomappings

In [5]:
biomappings_dd = defaultdict(lambda: defaultdict(dict))

for mapping in tqdm(biomappings.load_mappings(), unit_scale=True, unit="mapping"):
    source_prefix = mapping["source prefix"]
    source_resource = bioregistry.get_resource(source_prefix)
    source_id = source_resource.standardize_identifier(mapping["source identifier"])
    target_prefix = mapping["target prefix"]
    target_resource = bioregistry.get_resource(target_prefix)
    target_id = target_resource.standardize_identifier(mapping["target identifier"])
    biomappings_dd[source_prefix][target_prefix][source_id] = target_id
    biomappings_dd[target_prefix][source_prefix][target_id] = source_id

biomappings_dd = {k: dict(v) for k, v in biomappings_dd.items()}

  0%|          | 0.00/8.04k [00:00<?, ?mapping/s]

Predicted mappings from Biomappings

In [6]:
biomappings_predictions_dd = defaultdict(lambda: defaultdict(dict))

for mapping in tqdm(biomappings.load_predictions(), unit_scale=True, unit="mapping"):
    source_prefix = mapping["source prefix"]
    source_resource = bioregistry.get_resource(source_prefix)
    source_id = source_resource.standardize_identifier(mapping["source identifier"])
    target_prefix = mapping["target prefix"]
    target_resource = bioregistry.get_resource(target_prefix)
    target_id = target_resource.standardize_identifier(mapping["target identifier"])
    biomappings_predictions_dd[source_prefix][target_prefix][source_id] = target_id
    biomappings_predictions_dd[target_prefix][source_prefix][target_id] = source_id

biomappings_predictions_dd = {k: dict(v) for k, v in biomappings_predictions_dd.items()}

  0%|          | 0.00/41.9k [00:00<?, ?mapping/s]

## OBO Ontologies

In [7]:
@lru_cache
def get_graph(prefix: str, graph_uri=None):
    parse_results = bioontologies.get_obograph_by_prefix(prefix)
    if len(parse_results.graph_document.graphs) == 1:
        return parse_results.graph_document.graphs[0]
    if graph_uri is None:
        uris = sorted(graph.id for graph in parse_results.graph_document.graphs)
        raise ValueError(f"need a graph_uri for {prefix} since it has multiple graphs: {uris}.")
    return next(graph for graph in parse_results.graph_document.graphs if graph.id == graph_uri)

Get primary mappings from ontologies that can be parsed with ROBOT

In [8]:
def get_primary_mappings(prefix: str, graph_uri: str, external_prefix: str):
    cache_path = EVALUATION.join("mappings", name=f"{prefix}_{external_prefix}.json")
    if cache_path.is_file():
        d = json.loads(cache_path.read_text())
        return d["version"], d["mappings"]

    graph = get_graph(prefix, graph_uri)
    rv = {}
    for node in tqdm(
        graph.nodes,
        unit="node",
        unit_scale=True,
        desc=f"Extracting {external_prefix} from {prefix}",
    ):
        try:
            node_prefix, node_id = _compress_uri(node.id)
        except ValueError:
            continue

        if node_prefix is None or bioregistry.normalize_prefix(node_prefix) != prefix:
            continue

        for xref in node.xrefs:
            xref_prefix, xref_identifier = bioregistry.parse_curie(xref.val)
            if xref_prefix != external_prefix:
                continue
            rv[xref_identifier] = node_id

    version = graph.version or graph.version_iri
    d = {"mappings": rv, "version": version}
    cache_path.write_text(json.dumps(d, indent=2, sort_keys=True))
    return version, rv


config = [
    ("doid", "umls", "http://purl.obolibrary.org/obo/doid.owl"),
    ("doid", "mesh", "http://purl.obolibrary.org/obo/doid.owl"),
    ("doid", "mondo", "http://purl.obolibrary.org/obo/doid.owl"),
    ("doid", "efo", "http://purl.obolibrary.org/obo/doid.owl"),
    # ("doid", "omim", "http://purl.obolibrary.org/obo/doid.owl"),
    ("mondo", "umls", "http://purl.obolibrary.org/obo/mondo.owl"),
    ("mondo", "mesh", "http://purl.obolibrary.org/obo/mondo.owl"),
    ("mondo", "doid", "http://purl.obolibrary.org/obo/mondo.owl"),
    ("mondo", "efo", "http://purl.obolibrary.org/obo/mondo.owl"),
    # ("mondo", "omim", "http://purl.obolibrary.org/obo/mondo.owl"),
    ("efo", "mesh", "http://www.ebi.ac.uk/efo/efo.obo"),
    ("efo", "doid", "http://www.ebi.ac.uk/efo/efo.obo"),
    # ("efo", "cl", "http://www.ebi.ac.uk/efo/efo.obo"),
    # ("efo", "omim", "http://www.ebi.ac.uk/efo/efo.obo
    ("efo", "ccle", "http://www.ebi.ac.uk/efo/efo.obo"),
    ("hp", "mesh", "http://purl.obolibrary.org/obo/hp.owl"),
    # ("hp", "omim", "http://purl.obolibrary.org/obo/hp.owl"),
    ("go", "mesh", "http://purl.obolibrary.org/obo/go.owl"),
    ("go", "reactome", "http://purl.obolibrary.org/obo/go.owl"),
    ("go", "wikipathways", "http://purl.obolibrary.org/obo/go.owl"),
#     ("clo", "efo", "http://purl.obolibrary.org/obo/clo.owl"),
#     ("clo", "ccle", "http://purl.obolibrary.org/obo/clo.owl"),
#     ("clo", "depmap", "http://purl.obolibrary.org/obo/clo.owl"),
#     ("clo", "cellosaurus", "http://purl.obolibrary.org/obo/clo.owl"),
    ("uberon", "mesh", "http://purl.obolibrary.org/obo/uberon.owl"),
    ("cl", "efo", "http://purl.obolibrary.org/obo/cl.owl"),
    ("cl", "mesh", "http://purl.obolibrary.org/obo/cl.owl"),
    ("chebi", "mesh", "http://purl.obolibrary.org/obo/chebi.owl"),
    ("chebi", "ncit", "http://purl.obolibrary.org/obo/chebi.owl"),
]
for prefix, external, uri in config:
    version, primary = get_primary_mappings(prefix, uri, external)
    primary_dd[external][prefix] = primary
    n_primary = len(primary)

    bm = set(biomappings_dd.get(external, {}).get(prefix, {})).union(biomappings_dd.get(prefix, {}).get(external, {}))
    n_biomappings = len(bm)
    n_total = len(set(primary).union(bm))

    if not n_primary and n_biomappings:
        gain = float("inf")
    elif not n_primary and not n_biomappings:
        gain = "-"
    else:
        gain = round(100 * n_biomappings / n_primary, 1) if n_primary else None

    summary_rows.append(
        (
            prefix,
            (
                version.removeprefix(f"http://purl.obolibrary.org/obo/{prefix}/")
                .removeprefix("releases/")
                .removesuffix(f"/{prefix}.owl")
            ),
            external,
            n_primary,
            n_biomappings,
            n_total,
            gain,
        )
    )

# TODO other resources, like MeSH -> CAS?

## Non-OBO Resources via PyOBO

In [9]:
pyobo_configs = [
    ("cellosaurus", "efo", "CVCL_", "EFO_"),
    ("cellosaurus", "ccle", "CVCL_", ""),
    # ("cellosaurus", "cl", "CVCL_", ""),
]
for prefix, external, source_banana, target_banana in pyobo_configs:
    xrefs_df = pyobo.get_xrefs_df(prefix)
    xrefs_slim_df = xrefs_df[xrefs_df["target_ns"] == external]

    version = "unknown"  # FIXME, e.g., with bioversions.get_version(prefix)
    primary = primary_dd[external][prefix] = {
        target_id.removeprefix(target_banana): source_id.removeprefix(source_banana)
        for source_id, target_id in xrefs_slim_df[[f"{prefix}_id", "target_id"]].values
    }
    n_primary = len(primary)
    
    bm = set(biomappings_dd.get(external, {}).get(prefix, {})).union(biomappings_dd.get(prefix, {}).get(external, {}))
    n_biomappings = len(bm)
    n_total = len(set(primary).union(bm))

    if not n_primary and n_biomappings:
        gain = float("inf")
    elif not n_primary and not n_biomappings:
        gain = "-"
    else:
        gain = round(100 * n_biomappings / n_primary, 1) if n_primary else None

    summary_rows.append(
        (
            prefix,
            (
                version.removeprefix(f"http://purl.obolibrary.org/obo/{prefix}/")
                .removeprefix("releases/")
                .removesuffix(f"/{prefix}.owl")
            ),
            external,
            n_primary,
            n_biomappings,
            n_total,
            gain,
        )
    )

## Summary

In [10]:
summary_df = pd.DataFrame(
    summary_rows,
    columns=[
        "resource",
        "version",
        "external",
        "primary_xrefs",
        "biomappings_xrefs",
        "total_xrefs",
        "percentage_gain",
    ],
)
pd.option_context("display.max_rows", summary_df.shape[0])
summary_df

,resource,version,external,primary_xrefs,biomappings_xrefs,total_xrefs,percentage_gain
0,doid,2022-07-27,umls,6852,242,7033,3.5
1,doid,2022-07-27,mesh,3249,2611,5351,80.4
2,doid,2022-07-27,mondo,0,0,0,-
3,doid,2022-07-27,efo,131,122,253,93.1
4,doid,2022-07-27,omim,5539,0,5539,0.0
5,mondo,2022-08-01,umls,16751,0,16751,0.0
6,mondo,2022-08-01,mesh,8114,412,8342,5.1
7,mondo,2022-08-01,doid,9886,0,9886,0.0
8,mondo,2022-08-01,efo,2862,0,2862,0.0
9,mondo,2022-08-01,omim,9359,0,9359,0.0


# Assess Impact on Mapping Hetionet Datasources

## CTD Chemical-Gene Interactions

In [11]:
CTD_CHEMICAL_GENE_URL = "https://ctdbase.org/reports/CTD_chem_gene_ixns.tsv.gz"
ctd_header = [
    "chemical_name",
    "chemical_mesh_id",
    "chemical_cas",
    "gene_symbol",
    "gene_ncbigene_id",
    "gene_forms",
    "organism_name",
    "organism_ncbitaxon_id",
    "evidence",
    "interaction",
    "pubmed_ids",
]
ctd_gene_chemical_df = EVALUATION.ensure_csv(
    url=CTD_CHEMICAL_GENE_URL,
    read_csv_kwargs=dict(
        sep="\t",
        comment="#",
        header=None,
        dtype=str,
        keep_default_na=False,
        usecols=[1],
        squeeze=True,
    ),
)
ctd_gene_chemical_df.head()

0    C534883
1    C534883
2    C534883
3    C534883
4    C534883
Name: 1, dtype: object

In [12]:
result = Result.make(
    dataset="ctd-chemical-gene",
    source="mesh",
    target="chebi",
    datasource_identifiers=set(ctd_gene_chemical_df.tolist()),
    primary=primary_dd,
    secondary=biomappings_dd,
    tertiary=biomappings_predictions_dd,
)
result.print()
evaluation_results.append(result)

Missing                        Unmappable to chebi    % Unmappable
-----------------------------  ---------------------  --------------
Total in ctd-chemical-gene     14,337
Missing w/ mesh                14,337                 100.00%
Missing w/ mesh + BM.          13,064                 91.1%
Missing w/ mesh + BM. + Pred.  9,080                  63.3%


## CTD Chemical-Diseases

In [13]:
CTD_CHEMICAL_DISEASES_URL = "https://ctdbase.org/reports/CTD_chemicals_diseases.tsv.gz"
"""
    ChemicalName
    ChemicalID (MeSH identifier)
    CasRN (CAS Registry Number, if available)
    DiseaseName
    DiseaseID (MeSH or OMIM identifier)
    DirectEvidence ('|'-delimited list)
    InferenceGeneSymbol
    InferenceScore
    OmimIDs ('|'-delimited list)
    PubMedIDs ('|'-delimited list)
"""
ctd_chemical_diseases_df = EVALUATION.ensure_csv(
    url=CTD_CHEMICAL_DISEASES_URL,
    read_csv_kwargs=dict(
        sep="\t",
        comment="#",
        header=None,
        dtype=str,
        keep_default_na=False,
        usecols=[4],
        squeeze=True,
    ),
)
ctd_chemical_diseases_df.head()

0       MESH:D054198
1       MESH:D000230
2    MESH:D000077192
3       MESH:D000505
4       MESH:D013734
Name: 4, dtype: object

In [14]:
ctd_chemical_diseases_mesh = {
    x.split(":")[1] for x in ctd_chemical_diseases_df.tolist() if x.startswith("MESH")
}
ctd_chemical_diseases_omim = {
    x.split(":")[1] for x in ctd_chemical_diseases_df.tolist() if x.startswith("OMIM")
}

In [15]:
result = Result.make(
    dataset="ctd-gene-disease",
    source="mesh",
    target="doid",
    datasource_identifiers=ctd_chemical_diseases_mesh,
    primary=primary_dd,
    secondary=biomappings_dd,
    tertiary=biomappings_predictions_dd,
)
result.print()
evaluation_results.append(result)

Missing                        Unmappable to doid    % Unmappable
-----------------------------  --------------------  --------------
Total in ctd-gene-disease      5,821
Missing w/ mesh                3,342                 57.41%
Missing w/ mesh + BM.          2,785                 47.8%
Missing w/ mesh + BM. + Pred.  2,618                 45.0%


In [16]:
result = Result.make(
    dataset="ctd-gene-disease",
    source="mesh",
    target="mondo",
    datasource_identifiers=ctd_chemical_diseases_mesh,
    primary=primary_dd,
    secondary=biomappings_dd,
    tertiary=biomappings_predictions_dd,
)
result.print()
evaluation_results.append(result)

Missing                        Unmappable to mondo    % Unmappable
-----------------------------  ---------------------  --------------
Total in ctd-gene-disease      5,821
Missing w/ mesh                1,518                  26.08%
Missing w/ mesh + BM.          1,503                  25.8%
Missing w/ mesh + BM. + Pred.  1,458                  25.0%


# SIDER Side Effects

In [17]:
SIDE_EFFECTS_HEADER = [
    "STITCH_FLAT_ID",
    "STITCH_STEREO_ID",
    "UMLS CUI from Label",
    "MedDRA Concept Type",
    "UMLS CUI from MedDRA",
    "MedDRA Concept name",
]

side_effects_df = EVALUATION.ensure_csv(
    url=SIDER_URL,
    read_csv_kwargs=dict(
        dtype=str,
        header=None,
        names=SIDE_EFFECTS_HEADER,
    ),
)
side_effects_df

,STITCH_FLAT_ID,STITCH_STEREO_ID,UMLS CUI from Label,MedDRA Concept Type,UMLS CUI from MedDRA,MedDRA Concept name
0,CID100000085,CID000010917,C0000729,LLT,C0000729,Abdominal cramps
1,CID100000085,CID000010917,C0000729,PT,C0000737,Abdominal pain
2,CID100000085,CID000010917,C0000737,LLT,C0000737,Abdominal pain
3,CID100000085,CID000010917,C0000737,PT,C0687713,Gastrointestinal pain
4,CID100000085,CID000010917,C0000737,PT,C0000737,Abdominal pain
...,...,...,...,...,...,...
309844,CID171306834,CID071306834,C3203358,PT,C1145670,Respiratory failure
309845,CID171306834,CID071306834,C3665386,LLT,C3665386,Abnormal vision
309846,CID171306834,CID071306834,C3665386,PT,C3665347,Visual impairment
309847,CID171306834,CID071306834,C3665596,LLT,C3665596,Warts


In [18]:
result = Result.make(
    dataset="sider",
    source="umls",
    target="doid",
    datasource_identifiers=set(side_effects_df["UMLS CUI from Label"].unique()),
    primary=primary_dd,
    secondary=biomappings_dd,
    tertiary=biomappings_predictions_dd,
)
result.print()
evaluation_results.append(result)

Missing                        Unmappable to doid    % Unmappable
-----------------------------  --------------------  --------------
Total in sider                 5,868
Missing w/ umls                4,730                 80.61%
Missing w/ umls + BM.          4,724                 80.5%
Missing w/ umls + BM. + Pred.  4,619                 78.7%


## CCLE Achilles Cell Lines

In [19]:
# See https://depmap.org/portal/download/
# CCLE_ACHILLES_URL = "https://ndownloader.figshare.com/files/35020903"
CCLE_ACHILLES_URL = "/Users/cthoyt/Downloads/sample_info.csv"

ccle_achilles_df = pd.read_csv(CCLE_ACHILLES_URL)
ccle_achilles_df.head()

,DepMap_ID,cell_line_name,stripped_cell_line_name,CCLE_Name,alias,COSMICID,sex,source,RRID,WTSI_Master_Cell_ID,...,lineage_sub_subtype,lineage_molecular_subtype,default_growth_pattern,model_manipulation,model_manipulation_details,patient_id,parent_depmap_id,Cellosaurus_NCIt_disease,Cellosaurus_NCIt_id,Cellosaurus_issues
0,ACH-000016,SLR 21,SLR21,SLR21_KIDNEY,NaN,NaN,NaN,Academic lab,CVCL_V607,NaN,...,NaN,NaN,NaN,NaN,NaN,PT-JnARLB,NaN,Clear cell renal cell carcinoma,C4033,NaN
1,ACH-000032,MHH-CALL-3,MHHCALL3,MHHCALL3_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,NaN,NaN,Female,DSMZ,CVCL_0089,NaN,...,b_cell,NaN,NaN,NaN,NaN,PT-p2KOyI,NaN,Childhood B acute lymphoblastic leukemia,C9140,NaN
2,ACH-000033,NCI-H1819,NCIH1819,NCIH1819_LUNG,NaN,NaN,Female,Academic lab,CVCL_1497,NaN,...,NSCLC_adenocarcinoma,NaN,NaN,NaN,NaN,PT-9p1WQv,NaN,Lung adenocarcinoma,C3512,NaN
3,ACH-000043,Hs 895.T,HS895T,HS895T_FIBROBLAST,NaN,NaN,Female,ATCC,CVCL_0993,NaN,...,NaN,NaN,2D: adherent,NaN,NaN,PT-rTUVZQ,NaN,Melanoma,C3224,NaN
4,ACH-000049,HEK TE,HEKTE,HEKTE_KIDNEY,NaN,NaN,NaN,Academic lab,CVCL_WS59,NaN,...,NaN,NaN,NaN,immortalized,NaN,PT-qWYYgr,NaN,NaN,NaN,No information is available about this cell li...


In [20]:
result = Result.make(
    dataset="ccle-achilles",
    source="ccle",
    target="efo",
    datasource_identifiers=set(ccle_achilles_df["CCLE_Name"].unique()),
    primary=primary_dd,
    secondary=biomappings_dd,
    tertiary=biomappings_predictions_dd,
)
result.print()
evaluation_results.append(result)

Missing                        Unmappable to efo    % Unmappable
-----------------------------  -------------------  --------------
Total in ccle-achilles         1,837
Missing w/ ccle                1,837                100.00%
Missing w/ ccle + BM.          1,326                72.2%
Missing w/ ccle + BM. + Pred.  1,270                69.1%


## Summary

In [21]:
evaluation_df_rows = []
for result in evaluation_results:
    evaluation_df_rows.append((
        result.dataset,
        result.source,
        result.target,
        result.total,
        result.missing,
        round(100 * result.missing / result.total, 1),
        result.missing_biomappings,
        round(100 * result.missing_biomappings / result.total, 1),
        round(100 * (result.missing - result.missing_biomappings) / result.total, 1),
        result.missing_predictions,
        round(100 * result.missing_predictions / result.total, 1),
        round(100 * (result.missing - result.missing_predictions) / result.total, 1),
    ))
pd.DataFrame(
    evaluation_df_rows,
    columns=[
        "dataset",
        "source",
        "target",
        "total",
        "missing_w_primary",
        "m1 (%)",
        "missing_w_curations",
        "m2 (%)",
        "m2d",
        "missing_w_predictions",
        "m3 (%)",
        "m3d",
    ],
)

,dataset,source,target,total,missing_w_primary,m1 (%),missing_w_curations,m2 (%),m2d,missing_w_predictions,m3 (%),m3d
0,ctd-chemical-gene,mesh,chebi,14337,14337,100.0,13064,91.1,8.9,9080,63.3,36.7
1,ctd-gene-disease,mesh,doid,5821,3342,57.4,2785,47.8,9.6,2618,45.0,12.4
2,ctd-gene-disease,mesh,mondo,5821,1518,26.1,1503,25.8,0.3,1458,25.0,1.0
3,sider,umls,doid,5868,4730,80.6,4724,80.5,0.1,4619,78.7,1.9
4,ccle-achilles,ccle,efo,1837,1837,100.0,1326,72.2,27.8,1270,69.1,30.9
